### Import Modules

In [1]:
import hashlib
import os
import pickle
from urllib.request import urlretrieve
from tensorflow.examples.tutorials.mnist import input_data


import numpy as np
import tensorflow as tf
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import resample
from tqdm import tqdm
from zipfile import ZipFile

print('All modules imported.')

All modules imported.


### Download MNIST Data

In [2]:
def download(url, file):
    """
    Download file from <url>
    :param url: URL to file
    :param file: Local file path
    """
    if not os.path.isfile(file):
        print('Downloading ' + file + '...')
        urlretrieve(url, file)
        print('Download Finished')

# Download the training and test dataset.
download('https://s3.amazonaws.com/udacity-sdc/notMNIST_train.zip', 'notMNIST_train.zip')
download('https://s3.amazonaws.com/udacity-sdc/notMNIST_test.zip', 'notMNIST_test.zip')

# Make sure the files aren't corrupted
assert hashlib.md5(open('notMNIST_train.zip', 'rb').read()).hexdigest() == 'c8673b3f28f489e9cdf3a3d74e2ac8fa',\
        'notMNIST_train.zip file is corrupted.  Remove the file and try again.'
assert hashlib.md5(open('notMNIST_test.zip', 'rb').read()).hexdigest() == '5d3c7e653e63471c88df796156a9dfa9',\
        'notMNIST_test.zip file is corrupted.  Remove the file and try again.'

# Wait until you see that all files have been downloaded.
print('All files downloaded.')

All files downloaded.


In [3]:
def uncompress_features_labels(file):
    """
    Uncompress features and labels from a zip file
    :param file: The zip file to extract the data from
    """
    features = []
    labels = []

    with ZipFile(file) as zipf:
        # Progress Bar
        filenames_pbar = tqdm(zipf.namelist(), unit='files')
        
        # Get features and labels from all files
        for filename in filenames_pbar:
            # Check if the file is a directory
            if not filename.endswith('/'):
                with zipf.open(filename) as image_file:
                    image = Image.open(image_file)
                    image.load()
                    # Load image data as 1 dimensional array
                    # We're using float32 to save on memory space
                    feature = np.array(image, dtype=np.float32).flatten()

                # Get the the letter from the filename.  This is the letter of the image.
                label = os.path.split(filename)[1][0]

                features.append(feature)
                labels.append(label)
    return np.array(features), np.array(labels)

In [4]:
# Get the features and labels from the zip files
train_features, train_labels = uncompress_features_labels('notMNIST_train.zip')
test_features, test_labels = uncompress_features_labels('notMNIST_test.zip')


# Wait until you see that all features and labels have been uncompressed.
print('All features and labels uncompressed.')

100%|███████████████████████████████████████████████████████████████████████| 10001/10001 [00:03<00:00, 3329.28files/s]


All features and labels uncompressed.


In [5]:
mnist = input_data.read_data_sets("MNIST_Data/", one_hot=True)
X = tf.placeholder(tf.float32,[None,784])

W = tf.Variable(tf.zeros([784,10]))

b = tf.Variable(tf.zeros([10]))

Y = tf.nn.softmax(tf.matmul(tf.reshape(X,[-1,784]) , W) +b )

Y_ = tf.placeholder(tf.float32,[None,10])

cross_entropy = -tf.reduce_sum(Y_ * tf.log(Y))

#Display Variables
is_correct = tf.equal(tf.argmax(Y,1), tf.argmax(Y_,1))

accuracy = tf.reduce_mean(tf.cast(is_correct,tf.float32))


Extracting MNIST_Data/train-images-idx3-ubyte.gz
Extracting MNIST_Data/train-labels-idx1-ubyte.gz
Extracting MNIST_Data/t10k-images-idx3-ubyte.gz
Extracting MNIST_Data/t10k-labels-idx1-ubyte.gz


In [6]:
training_rate = 0.003
epochs = 1000
batch_size = 100

optimizer = tf.train.GradientDescentOptimizer(training_rate)

train_step = optimizer.minimize(cross_entropy)

In [7]:
#training Loop
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for i in range(epochs):
    batch_X,batch_Y = mnist.train.next_batch(batch_size)
    
    sess.run(train_step,feed_dict={X: batch_X, Y_: batch_Y})
    
    a,c = sess.run([accuracy,cross_entropy], feed_dict={X: batch_X, Y_: batch_Y})
    
   
    
    
a,c = sess.run([accuracy,cross_entropy], feed_dict={X: batch_X, Y_: batch_Y})
print ("Test accuracy: {}" .format(a))

Test accuracy: 0.9399999976158142
